In [1]:
import pandas as pd
import os
import requests
from zipfile import ZipFile
from dbfread import DBF
from utils import download_csv

In [2]:
variaveis_observa = pd.read_csv(os.path.join('original_data', 'variaveis_observa.csv'),
                               sep=';', encoding='utf-8')

In [3]:
vars_names = variaveis_observa['Nome'].unique()

In [4]:
var_alunos = [var for var in vars_names if 'alun' in var.lower()]

In [5]:
var_alunos

['V0008-Alunos rede municipal - sexo feminino',
 'V0009-Alunos da rede municipal de ensino',
 'V0010-Alunos rede municipal - sexo masculino',
 'V0033-Alunos da rede municipal de ensino - raça/cor amarela',
 'V0034-Alunos da rede municipal de ensino da raça/cor branca',
 'V0035-Alunos da rede municipal de ensino da raça/cor indígena',
 'V0036-Alunos da rede municipal de ensino da raça/cor parda',
 'V0042-Alunos da rede municipal de ensino da raça/cor preta',
 'V0044-Alunos do Ensino Fundamental que utilizam Transporte Escolar Gratuito',
 'V0061-Alunos beneficiados com a distribuição de Uniformes e Materiais Escolares na Educação Infantil na rede municipal',
 'V0062-Alunos beneficiados pelo Programa de Alimentação Escolar',
 'V0070-Alunos beneficiados com a distribuição de Uniformes e Materiais Escolares no Ensino Fundamental na rede municipal',
 'V0081-Alunos do Ensino Fundamental da rede municipal de ensino matriculados com idade acima da recomendada para a série (idade recomendada +2)

In [6]:
variaveis = pd.DataFrame(data=[], columns=['variavel', 'ano', 'regiao', 'valor'])

In [7]:
variaveis

,variavel,ano,regiao,valor


In [8]:
link_perfil_educandos = 'http://dados.prefeitura.sp.gov.br/dataset/b9c5faef-1a61-4814-b89e-bd7a73154ebc/resource/0a422138-92a3-422b-ab71-864d88634a50/download/idadeserieneeracadez22.csv'

In [9]:
download_csv(link_perfil_educandos, 'perfil_educandos.csv')

In [10]:
df = pd.read_csv(os.path.join('original_data', 'perfil_educandos.csv'),
                sep=';')

In [11]:
df.head()

,DRE,CODESC,TIPOESC,NOMESC,DISTRITO,SETOR,ANO,REDE,MODAL,DESCSERIE,PERIODO,TURNO,DESCTURNO,SEXO,IDADE,NEE,RACA,Qtde,DATABASE
0,G,86,EMEI,PAULO CAMILHIER FLORENCANO,GUAIANASES,3103,2022,DIR,PRE,INFANTIL UNIF,DIURNO,1,Manhã,F,5,NAO POSSUI,BRANCA,3,30/12/2022 00:00
1,G,86,EMEI,PAULO CAMILHIER FLORENCANO,GUAIANASES,3103,2022,DIR,PRE,INFANTIL UNIF,DIURNO,1,Manhã,F,5,NAO POSSUI,NÃO DECLARADA,1,30/12/2022 00:00
2,G,86,EMEI,PAULO CAMILHIER FLORENCANO,GUAIANASES,3103,2022,DIR,PRE,INFANTIL UNIF,DIURNO,1,Manhã,F,5,NAO POSSUI,PARDA,6,30/12/2022 00:00
3,G,86,EMEI,PAULO CAMILHIER FLORENCANO,GUAIANASES,3103,2022,DIR,PRE,INFANTIL UNIF,DIURNO,1,Manhã,F,6,NAO POSSUI,BRANCA,2,30/12/2022 00:00
4,G,86,EMEI,PAULO CAMILHIER FLORENCANO,GUAIANASES,3103,2022,DIR,PRE,INFANTIL UNIF,DIURNO,1,Manhã,F,6,NAO POSSUI,PARDA,1,30/12/2022 00:00


In [12]:
df['ANO'].unique()

array([2022])

In [13]:
df.keys()

Index(['DRE', 'CODESC', 'TIPOESC', 'NOMESC', 'DISTRITO', 'SETOR', 'ANO',
       'REDE', 'MODAL', 'DESCSERIE', 'PERIODO', 'TURNO', 'DESCTURNO', 'SEXO',
       'IDADE', 'NEE', 'RACA', 'Qtde', 'DATABASE'],
      dtype='object')

In [14]:
df['RACA'].unique()

array(['BRANCA', 'NÃO DECLARADA', 'PARDA', 'RECUSOU INFORMA', 'PRETA',
       'AMARELA', 'INDIGENA'], dtype=object)

In [15]:
raca_por_dist = df.groupby(['DISTRITO', 'RACA'])[['Qtde']].sum().reset_index()

In [16]:
raca_por_dist.head()

,DISTRITO,RACA,Qtde
0,AGUA RASA,AMARELA,39
1,AGUA RASA,BRANCA,2573
2,AGUA RASA,INDIGENA,27
3,AGUA RASA,NÃO DECLARADA,2
4,AGUA RASA,PARDA,954


In [17]:
de_para_var_racas = {
    'V0033-Alunos da rede municipal de ensino - raça/cor amarela' : 'AMARELA',
     'V0034-Alunos da rede municipal de ensino da raça/cor branca' : 'BRANCA',
     'V0035-Alunos da rede municipal de ensino da raça/cor indígena' : 'INDIGENA',
     'V0036-Alunos da rede municipal de ensino da raça/cor parda' : 'PARDA',
     'V0042-Alunos da rede municipal de ensino da raça/cor preta' : 'PRETA'
}

In [18]:
for var, raca in de_para_var_racas.items():
    
    df_raca = raca_por_dist[raca_por_dist['RACA']==raca].copy()
    df_raca['variavel'] = var
    
    df_raca.drop('RACA', axis=1, inplace=True)
    df_raca['ano'] = 2022
    df_raca.rename({'DISTRITO' : 'regiao',
                   'Qtde' : 'valor'}, axis=1, inplace=True)
    
    variaveis = pd.concat([variaveis, df_raca])

In [19]:
variaveis

,variavel,ano,regiao,valor
0,V0033-Alunos da rede municipal de ensino - raç...,2022,AGUA RASA,39
7,V0033-Alunos da rede municipal de ensino - raç...,2022,ALTO DE PINHEIROS,8
12,V0033-Alunos da rede municipal de ensino - raç...,2022,ANHANGUERA,73
19,V0033-Alunos da rede municipal de ensino - raç...,2022,ARICANDUVA,14
26,V0033-Alunos da rede municipal de ensino - raç...,2022,ARTUR ALVIM,79
...,...,...,...,...
635,V0042-Alunos da rede municipal de ensino da ra...,2022,VILA MARIANA,83
642,V0042-Alunos da rede municipal de ensino da ra...,2022,VILA MATILDE,447
649,V0042-Alunos da rede municipal de ensino da ra...,2022,VILA MEDEIROS,361
656,V0042-Alunos da rede municipal de ensino da ra...,2022,VILA PRUDENTE,147


In [20]:
df['SEXO'].unique()

array(['F', 'M'], dtype=object)

In [21]:
por_sexo = df.groupby(['DISTRITO', 'SEXO'])[['Qtde']].sum().reset_index()

In [22]:
por_sexo

,DISTRITO,SEXO,Qtde
0,AGUA RASA,F,1880
1,AGUA RASA,M,2078
2,ALTO DE PINHEIROS,F,167
3,ALTO DE PINHEIROS,M,172
4,ANHANGUERA,F,5802
...,...,...,...
187,VILA MEDEIROS,M,5026
188,VILA PRUDENTE,F,2724
189,VILA PRUDENTE,M,3022
190,VILA SONIA,F,4421


In [23]:
de_para_var_sexo = {
    'V0010-Alunos rede municipal - sexo masculino' : 'M',
    'V0008-Alunos rede municipal - sexo feminino' : 'F'
}

In [24]:
for var, sexo in de_para_var_sexo.items():
    
    df_sexo = por_sexo[por_sexo['SEXO']==sexo].copy()
    df_sexo['variavel'] = var
    
    df_sexo.drop('SEXO', axis=1, inplace=True)
    df_sexo['ano'] = 2022
    df_sexo.rename({'DISTRITO' : 'regiao',
                   'Qtde' : 'valor'}, axis=1, inplace=True)
    
    variaveis = pd.concat([variaveis, df_sexo])

In [25]:
variaveis

,variavel,ano,regiao,valor
0,V0033-Alunos da rede municipal de ensino - raç...,2022,AGUA RASA,39
7,V0033-Alunos da rede municipal de ensino - raç...,2022,ALTO DE PINHEIROS,8
12,V0033-Alunos da rede municipal de ensino - raç...,2022,ANHANGUERA,73
19,V0033-Alunos da rede municipal de ensino - raç...,2022,ARICANDUVA,14
26,V0033-Alunos da rede municipal de ensino - raç...,2022,ARTUR ALVIM,79
...,...,...,...,...
182,V0008-Alunos rede municipal - sexo feminino,2022,VILA MARIANA,997
184,V0008-Alunos rede municipal - sexo feminino,2022,VILA MATILDE,3419
186,V0008-Alunos rede municipal - sexo feminino,2022,VILA MEDEIROS,4775
188,V0008-Alunos rede municipal - sexo feminino,2022,VILA PRUDENTE,2724


In [26]:
total_por_dist = df.groupby('DISTRITO')[['Qtde']].sum().reset_index()

In [27]:
total_por_dist

,DISTRITO,Qtde
0,AGUA RASA,3958
1,ALTO DE PINHEIROS,339
2,ANHANGUERA,11386
3,ARICANDUVA,5078
4,ARTUR ALVIM,9618
...,...,...
91,VILA MARIANA,2093
92,VILA MATILDE,6994
93,VILA MEDEIROS,9801
94,VILA PRUDENTE,5746


In [28]:
total_por_dist['ano'] = 2022
total_por_dist.rename({'DISTRITO' : 'regiao',
                   'Qtde' : 'valor'}, axis=1, inplace=True)

In [29]:
total_por_dist

,regiao,valor,ano
0,AGUA RASA,3958,2022
1,ALTO DE PINHEIROS,339,2022
2,ANHANGUERA,11386,2022
3,ARICANDUVA,5078,2022
4,ARTUR ALVIM,9618,2022
...,...,...,...
91,VILA MARIANA,2093,2022
92,VILA MATILDE,6994,2022
93,VILA MEDEIROS,9801,2022
94,VILA PRUDENTE,5746,2022


In [30]:
total_por_dist['variavel'] = 'V0009-Alunos da rede municipal de ensino'

In [31]:
total_por_dist.head()

,regiao,valor,ano,variavel
0,AGUA RASA,3958,2022,V0009-Alunos da rede municipal de ensino
1,ALTO DE PINHEIROS,339,2022,V0009-Alunos da rede municipal de ensino
2,ANHANGUERA,11386,2022,V0009-Alunos da rede municipal de ensino
3,ARICANDUVA,5078,2022,V0009-Alunos da rede municipal de ensino
4,ARTUR ALVIM,9618,2022,V0009-Alunos da rede municipal de ensino


In [32]:
variaveis = pd.concat([variaveis, total_por_dist])

In [33]:
len(variaveis)

765

In [34]:
df['DESCTURNO'].unique()

array(['Manhã', 'Tarde', 'Intermediário', 'Integral', 'Vespertino',
       'Noite'], dtype=object)

In [35]:
integral = df[df['DESCTURNO']=='Integral'].reset_index(drop=True)

In [36]:
integral.keys()

Index(['DRE', 'CODESC', 'TIPOESC', 'NOMESC', 'DISTRITO', 'SETOR', 'ANO',
       'REDE', 'MODAL', 'DESCSERIE', 'PERIODO', 'TURNO', 'DESCTURNO', 'SEXO',
       'IDADE', 'NEE', 'RACA', 'Qtde', 'DATABASE'],
      dtype='object')

In [37]:
for serie in df['DESCSERIE'].unique():
    print(serie)

INFANTIL UNIF
C.I 1.ANO F9
C.I 2.ANO F9
C.I 3.ANO F9
C.II 4.ANO F9
C.II 5.ANO F9
C.II 6.ANO F9
C.III 7.ANO F9
C.III 8.ANO F9
C.III 9.ANO F9
ATCOMP
EEC.I 1.ANO F9
EEC.I 2.ANO F9
EEC.I 3.ANO F9
EEC.II 4.ANO F9
EEC.II 5.ANO F9
EEC.II 6.ANO F9
EEC.III 7.ANO F9
EEC.III 8.ANO F9
EEC.III 9.ANO F9
PAP
EJA COMPL II
EJA COMPL I
EJA FINAL I
EJA FINAL II
INFANTIL I E II ESP
OSL
OIE
EJA MII
EJA MIII
EJA MIV
EJA BASICA II
RP AUTORAL PORTUGUES
RP AUTORAL MAT
RP AUTORAL CIENC HUM
RP AUTORAL CIENC NAT
EJA ALF II
EJA BASICA I
EJA ALF I
SAAI COMPL DI
SAAI COMPL DF
1A.SER.E.M.
2A.SER.E.M.
3A.SER E.M
MI - ED PROF
MINI GRUPO II
EEC.I Biling
PAEE COLAB
BERCARIO I
BERCARIO II
MINI GRUPO I
EE PRE
SAAI COMPL DA
EE 1S MED
EE 3S MED
EE 2S MED
EJA MI
1A.C.NORMAL
2A.C.NORMAL
3A.C.NORMAL
4A.C.NORMAL
CMCT
MI CIEJA
MII CIEJA
MIII CIEJA
MIV CIEJA
INFANTIL I E II DIF
MII - ED PROF
MIII - ED PROF
MINI G UNIF
INFANTIL I E II CEI
MOVA
CONVEE
CELP ESPANHOL
CELP INGLES
CELP ITALIANO
CELP ALEMAO
CELP FRANCES


In [38]:
iniciais = range(1, 6)
iniciais = [f'{i}.ANO' for i in iniciais]

In [39]:
iniciais

['1.ANO', '2.ANO', '3.ANO', '4.ANO', '5.ANO']

In [40]:
finais = range(6, 10)
finais = [f'{i}.ANO' for i in finais]

In [41]:
finais

['6.ANO', '7.ANO', '8.ANO', '9.ANO']

In [42]:
def definir_fund(serie:str)->str:
    
    for ano in iniciais:
        if ano in serie:
            return 'fund1'
    
    for ano in finais:
        if ano in serie:
            return 'fund2'
    
    return 'nao'

In [43]:
df['fundamental'] = df['DESCSERIE'].apply(definir_fund)

In [44]:
df['is_fundamental'] = df['fundamental']!='nao'

In [45]:
df[df['is_fundamental']]['DESCSERIE'].unique()

array(['C.I 1.ANO F9', 'C.I 2.ANO F9', 'C.I 3.ANO F9', 'C.II 4.ANO F9',
       'C.II 5.ANO F9', 'C.II 6.ANO F9', 'C.III 7.ANO F9',
       'C.III 8.ANO F9', 'C.III 9.ANO F9', 'EEC.I 1.ANO F9',
       'EEC.I 2.ANO F9', 'EEC.I 3.ANO F9', 'EEC.II 4.ANO F9',
       'EEC.II 5.ANO F9', 'EEC.II 6.ANO F9', 'EEC.III 7.ANO F9',
       'EEC.III 8.ANO F9', 'EEC.III 9.ANO F9'], dtype=object)

In [46]:
de_para_fund = {
    'V0609-Número de alunos matriculados em tempo integral no Ensino Fundamental - anos iniciais' : 'fund1',
     'V0610-Número de alunos matriculados em tempo integral no Ensino Fundamental - anos finais' : 'fund2'}

In [47]:
integral['fundamental'] = integral['DESCSERIE'].apply(definir_fund)

In [48]:
integral_por_fund = integral.groupby(['DISTRITO', 'fundamental'])[['Qtde']].sum().reset_index()

In [49]:
integral_por_fund

,DISTRITO,fundamental,Qtde
0,AGUA RASA,fund1,32
1,AGUA RASA,fund2,188
2,AGUA RASA,nao,1364
3,ALTO DE PINHEIROS,nao,339
4,ANHANGUERA,fund1,90
...,...,...,...
188,VILA MEDEIROS,fund2,77
189,VILA MEDEIROS,nao,4328
190,VILA PRUDENTE,nao,2529
191,VILA SONIA,fund1,183


In [50]:
integral_por_fund = integral_por_fund[integral_por_fund['fundamental']!='nao'].reset_index(drop=True)

In [51]:
integral_por_fund.head()

,DISTRITO,fundamental,Qtde
0,AGUA RASA,fund1,32
1,AGUA RASA,fund2,188
2,ANHANGUERA,fund1,90
3,ANHANGUERA,fund2,114
4,BRASILANDIA,fund1,258


In [52]:
for var, fund in de_para_fund.items():
    
    df_fund = integral_por_fund[integral_por_fund['fundamental']==fund].copy()
    df_fund['variavel'] = var
    
    df_fund.drop('fundamental', axis=1, inplace=True)
    df_fund['ano'] = 2022
    df_fund.rename({'DISTRITO' : 'regiao',
                   'Qtde' : 'valor'}, axis=1, inplace=True)
    
    variaveis = pd.concat([variaveis, df_fund])

In [53]:
variaveis.tail()

,variavel,ano,regiao,valor
83,V0610-Número de alunos matriculados em tempo i...,2022,VILA CURUCA,265
87,V0610-Número de alunos matriculados em tempo i...,2022,VILA JACUI,157
89,V0610-Número de alunos matriculados em tempo i...,2022,VILA LEOPOLDINA,324
91,V0610-Número de alunos matriculados em tempo i...,2022,VILA MARIA,221
95,V0610-Número de alunos matriculados em tempo i...,2022,VILA MEDEIROS,77


In [54]:
len(variaveis)

862

In [55]:
link_distritos = '	https://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadArquivo.aspx?orig=DownloadCamadas&arq=01_Limites Administrativos\\Distrito\\Shapefile\\SIRGAS_SHP_distrito&arqTipo=Shapefile'

with requests.get(link_distritos) as r:
    bytes_data=  r.content
    
with open(os.path.join('original_data', 'distritos.zip'), 'wb') as f:
    f.write(bytes_data)
    
zipfile_ = ZipFile(os.path.join('original_data', 'distritos.zip'), 'r') 

In [56]:
zipfile_.filelist

[<ZipInfo filename='SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.dbf' compress_type=deflate external_attr=0x20 file_size=49026 compress_size=2885>,
 <ZipInfo filename='SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.prj' compress_type=deflate external_attr=0x20 file_size=442 compress_size=281>,
 <ZipInfo filename='SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.shp' compress_type=deflate external_attr=0x20 file_size=2489012 compress_size=1835166>,
 <ZipInfo filename='SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.shx' compress_type=deflate external_attr=0x20 file_size=868 compress_size=675>]

In [57]:
zipfile_.extract('SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.dbf', path='original_data')

'original_data/SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.dbf'

In [58]:
dbf_path = os.path.join(os.path.join('original_data','SIRGAS_SHP_distrito'), 'SIRGAS_SHP_distrito.dbf')

In [59]:
dists = []
for record in DBF(dbf_path):
     dists.append(record)

In [60]:
df_dists = pd.DataFrame(dists)

In [61]:
df_dists

,ds_codigo,ds_nome,ds_sigla,ds_cd_sub,ds_subpref,ds_areamt,ds_areakmt
0,51,MANDAQUI,MAN,05,SANTANA-TUCURUVI,1.324786e+07,13.25
1,52,MARSILAC,MAR,20,PARELHEIROS,2.081957e+08,208.20
2,32,MOEMA,MOE,12,VILA MARIANA,9.079516e+06,9.08
3,31,GUAIANASES,GUA,28,GUAIANASES,8.938527e+06,8.94
4,33,IGUATEMI,IGU,30,SAO MATEUS,1.958356e+07,19.58
...,...,...,...,...,...,...,...
91,17,CAMPO LIMPO,CLM,17,CAMPO LIMPO,1.259634e+07,12.60
92,41,JAGUARE,JRE,08,LAPA,6.579876e+06,6.58
93,42,JARAGUA,JAR,02,PIRITUBA-JARAGUA,2.833031e+07,28.33
94,44,JARDIM HELENA,JDH,23,SAO MIGUEL,9.148559e+06,9.15


In [62]:
set(variaveis['regiao'])-set(df_dists['ds_nome'])

set()

In [63]:
len(variaveis)

862

In [64]:
dists_subs = df_dists[['ds_nome', 'ds_subpref']]

In [65]:
tmp_subs = pd.merge(variaveis, dists_subs, how='left', left_on='regiao', right_on='ds_nome')

In [66]:
tmp_subs['ds_subpref'].isnull().any()

False

In [67]:
tmp_subs = tmp_subs.groupby(['variavel', 'ds_subpref'])[['valor']].sum().reset_index()

In [68]:
tmp_subs.head()

,variavel,ds_subpref,valor
0,V0008-Alunos rede municipal - sexo feminino,ARICANDUVA-FORMOSA-CARRAO,7303
1,V0008-Alunos rede municipal - sexo feminino,BUTANTA,21277
2,V0008-Alunos rede municipal - sexo feminino,CAMPO LIMPO,47184
3,V0008-Alunos rede municipal - sexo feminino,CAPELA DO SOCORRO,31917
4,V0008-Alunos rede municipal - sexo feminino,CASA VERDE-CACHOEIRINHA,13049


In [69]:
tmp_subs['ano'] = 2022

In [70]:
tmp_subs.rename({'ds_subpref' : 'regiao'}, axis=1, inplace=True)

In [71]:
len(variaveis)

862

In [72]:
len(tmp_subs)

312

In [73]:
variaveis = pd.concat([variaveis, tmp_subs])

In [74]:
variaveis.to_csv('variaveis_perfil_alunos_educacao.csv', sep=';', encoding='utf-8', index=False)